<a href="https://colab.research.google.com/github/ThomasDougherty/stylish-people/blob/main/stylish_people.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome!
## This is a collaborative notebook for class specific style transfers


In [114]:
#@title Select options

PEOPLE = True #@param {type:"boolean"}
SKATEBOARD = False #@param {type:"boolean"}
OTHER_CLASSES = '' #@param {type:"string"}
STYLE = "mosaic" #@param ["candy", "mosaic", "rain_princess", "udnie"]

INVERSE = False #@param {type:"boolean"}
CLASS_SPECIFIC = True #@param {type:"boolean"}

STYLE_PTH = "saved_models/" + STYLE + ".pth"

objects = []
if PEOPLE:
  objects.append("person")
if SKATEBOARD:
  objects.append("skateboard")
if OTHER_CLASSES:
  OTHER_CLASSES = OTHER_CLASSES.replace(" ", "")
  other_list = OTHER_CLASSES.split(",")

In [115]:
#@title Select file(s)

from google.colab import files
files_list = files.upload()

import sys
!pip install filetype
import filetype

media_formats = ["video", "image"]
for key in files_list:
  kind = filetype.guess(key)
  file_type = kind.mime.split("/")[0]
  if file_type not in media_formats:
    print("Unrecognized media file: " + key)
    sys.exit()
  print(kind.mime)
  print(key)
  file_name = key

Saving IMG_6296.mov to IMG_6296.mov
video/quicktime
IMG_6296.mov


In [116]:
#@title Collecting stuff...
# import some common 
import json
import os
import re
import random
import subprocess

import cv2
import filetype
import numpy as np
from tqdm import tqdm

!pip install pyyaml==5.1
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
import torchvision
from torchvision import transforms

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html &> /dev/null
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Installing pretrained model weights
import zipfile

def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(path=dest_dir)

try:
    from torch.utils.model_zoo import _download_url_to_file
except ImportError:
    try:
        from torch.hub import download_url_to_file as _download_url_to_file
    except ImportError:
        from torch.hub import _download_url_to_file

_download_url_to_file('https://www.dropbox.com/s/lrvwfehqdcxoza8/saved_models.zip?dl=1', 'saved_models.zip', None, True)
unzip('saved_models.zip', '.')

# Creating directories
!mkdir -p finished_products

FINISHED_PRODUCTS = "finished_products/"
TEMP_DIR = "temp_imgs/"
ORG_DIR = "temp_imgs/org_imgs/"
STYLE_DIR = "temp_imgs/style_imgs/"
MASKS_DIR = "temp_imgs/masks_imgs/"
FINAL_DIR = "temp_imgs/final_imgs/"

In [117]:
#@title Utilities

def create_mask(outputs, num_class_list, img):
  '''Creates binary masks of detected objects'''
  class_pred = outputs["instances"].pred_classes.detach().cpu().numpy()
  masks_pred = outputs["instances"].pred_masks.detach().cpu().numpy()
  bin_mask = np.zeros(img.shape[:2])
  if len(class_pred) != 0:
    for cnt, c in enumerate(class_pred):
      if c in num_class_list:
        bin_mask += masks_pred[cnt]*1
  bin_mask[bin_mask > 0] = 1
  return bin_mask

def extract_frames(file_name):
  '''Extract frames from video "file_name" '''
  vidcap = cv2.VideoCapture(file_name)
  success,image = vidcap.read()
  frame_num = 0
  img_list = []
  base_names = []
  while success:
      cv2.imwrite(ORG_DIR + "frame_{0:05}.png".format(frame_num), image)
      success,image = vidcap.read() 
      base_names.append("frame_{0:05}.png".format(frame_num))       
      frame_num += 1            
  base_names.sort()
  return base_names, vidcap

def final_imgs(base_names):
  '''Combines the style and mask images to create final frames.'''
  for base in tqdm(base_names):
      if CLASS_SPECIFIC:
        frame_img = cv2.imread(ORG_DIR + base)
        style_img = cv2.imread(STYLE_DIR + base)
        mask_img = cv2.imread(MASKS_DIR + base)

        if style_img.shape != frame_img.shape:
          f_w, f_h, c = frame_img.shape
          style_img = cv2.resize(style_img, (f_h, f_w), interpolation = cv2.INTER_AREA)
        if INVERSE:
          frame_img[mask_img==0] = 0
          style_img[mask_img==1] = 0
        else:
          frame_img[mask_img==1] = 0
          style_img[mask_img==0] = 0
        final_img = frame_img + style_img
        cv2.imwrite(FINAL_DIR + base, final_img)
      else:
        style_img = cv2.imread(STYLE_DIR + base)
        cv2.imwrite(FINAL_DIR + base, style_img)
  return

def create_vid(base_names, vidcap, save_name):
  '''Creates video from final frames'''
  img = cv2.imread(ORG_DIR + base_names[0])
  height, width, layers = img.shape
  size = (width,height)

  fps = vidcap.get(cv2.CAP_PROP_FPS)
  out = cv2.VideoWriter(TEMP_DIR + 'vid_nosound.mp4',  #Provide a file to write the video to
                        cv2.VideoWriter_fourcc(*'DIVX'),
                        round(fps),                                        
                        size)
  for base in tqdm(base_names):
    img = cv2.imread(FINAL_DIR + base)
    out.write(img)
  out.release()

  command = "ffmpeg -i temp_imgs/final_imgs/frame_%05d.png -c:v libx264 -vf fps=" + str(round(fps)) + " " + TEMP_DIR + 'vid_nosound.mp4'
  subprocess.call(command, shell=True)

  command = "ffmpeg -i " + file_name + " -ab 160k -ac 2 -ar 44100 -vn " + TEMP_DIR + "audio.wav"
  subprocess.call(command, shell=True)

  command = "ffmpeg -i " + TEMP_DIR + 'vid_nosound.mp4' + " -i " + TEMP_DIR + "audio.wav -c:v copy -c:a aac " + TEMP_DIR + "vid_b_conv.mp4"
  subprocess.call(command, shell=True)

  if save_name is None:
    save_name = file_name + "style"
  command = "ffmpeg -i " + TEMP_DIR + "vid_b_conv.mp4 -vcodec libx264 -profile:v main -level 3.1 -preset medium -crf 23 -x264-params ref=4 -acodec copy -movflags +faststart  " + FINISHED_PRODUCTS + save_name + ".mp4"
  subprocess.call(command, shell=True)
  return

def remove_temp():
  ! rm -r temp_imgs
  return

def create_temp():
  !mkdir -p temp_imgs
  !mkdir -p temp_imgs/org_imgs/
  !mkdir -p temp_imgs/style_imgs/
  !mkdir -p temp_imgs/masks_imgs/
  !mkdir -p temp_imgs/final_imgs/
  return

def create_name(key):
  base = key.split(".")[0] + "_" + STYLE
  if INVERSE:
    base = base + "_INVERSE"
  if not CLASS_SPECIFIC:
    base = base + "_NOTSPECIFIC"
  for ob in objects:
    base = base + "_" + ob
  return base + "_" + timestamp


In [118]:
#@title Style transfer models
class TransformerNet(torch.nn.Module):
    def __init__(self):
        super(TransformerNet, self).__init__()
        # Initial convolution layers
        self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(32, affine=True)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = torch.nn.InstanceNorm2d(64, affine=True)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = torch.nn.InstanceNorm2d(128, affine=True)
        # Residual layers
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)
        # Upsampling Layers
        self.deconv1 = UpsampleConvLayer(128, 64, kernel_size=3, stride=1, upsample=2)
        self.in4 = torch.nn.InstanceNorm2d(64, affine=True)
        self.deconv2 = UpsampleConvLayer(64, 32, kernel_size=3, stride=1, upsample=2)
        self.in5 = torch.nn.InstanceNorm2d(32, affine=True)
        self.deconv3 = ConvLayer(32, 3, kernel_size=9, stride=1)
        # Non-linearities
        self.relu = torch.nn.ReLU()

    def forward(self, X):
        y = self.relu(self.in1(self.conv1(X)))
        y = self.relu(self.in2(self.conv2(y)))
        y = self.relu(self.in3(self.conv3(y)))
        y = self.res1(y)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.relu(self.in4(self.deconv1(y)))
        y = self.relu(self.in5(self.deconv2(y)))
        y = self.deconv3(y)
        return y


class ConvLayer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out


class ResidualBlock(torch.nn.Module):
    """ResidualBlock
    introduced in: https://arxiv.org/abs/1512.03385
    recommended architecture: http://torch.ch/blog/2016/02/04/resnets.html
    """

    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out


class UpsampleConvLayer(torch.nn.Module):
    """UpsampleConvLayer
    Upsamples the input and then does a convolution. This method gives better results
    compared to ConvTranspose2d.
    ref: http://distill.pub/2016/deconv-checkerboard/
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out

In [119]:
#@title Engine

def run_engine(base_names):
  '''Runs the stylization and detection models on frames'''
  # stylization setup
  content_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Lambda(lambda x: x.mul(255))
  ])
  with torch.no_grad():
      style_model = TransformerNet()
      state_dict = torch.load(STYLE_PTH)
      # remove saved deprecated running_* keys in InstanceNorm from the checkpoint
      for k in list(state_dict.keys()):
          if re.search(r'in\d+\.running_(mean|var)$', k):
              del state_dict[k]
      style_model.load_state_dict(state_dict)
      style_model.to(device='cuda:0')
  style_imgs = []

  if CLASS_SPECIFIC:
    # Detectron2 setup
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)

    # get class list 
    img = cv2.imread(ORG_DIR + base_names[0])
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    class_list = v.metadata.thing_classes

    # convert object classes to index
    num_class_list = []
    for ob in objects:
      num_class_list.append(class_list.index(ob))

  for base in tqdm(base_names):
    img = cv2.imread(ORG_DIR + base)

    if CLASS_SPECIFIC:
      outputs = predictor(img)
      binary_mask = create_mask(outputs, num_class_list, img)
      cv2.imwrite(MASKS_DIR + base, binary_mask)

    content_image = img
    content_image = content_transform(content_image)
    content_image = content_image.unsqueeze(0).to(device='cuda:0')
    with torch.no_grad():
      style_img = style_model(content_image)
      style_img = torch.squeeze(style_img)
      style_img = style_img.permute(1, 2, 0).detach().cpu().numpy()
    cv2.imwrite(STYLE_DIR + base, style_img) 
  return

In [120]:
#@title Main
import datetime
! pip install pyheif
import pyheif
import PIL
from PIL import Image
import numpy as np

is_video = None
for key in files_list:
  print("Stylizing " + key)
  create_temp()
  kind = filetype.guess(key)
  file_type = kind.mime.split("/")[0]
  if file_type == "video":
    is_video = True
  else:
    is_video = False
  file_name = key

  timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

  save_name = create_name(key)

  # if video extract frames, else move image into ORG_DIR
  
  if is_video:
    print("Extracting Frames...")
    base_names, vidcap = extract_frames(file_name)
  else:
    if file_name.split(".")[1].lower() == 'heic':
      heif_file = pyheif.read(file_name)
      image = Image.frombytes(
          heif_file.mode, 
          heif_file.size, 
          heif_file.data,
          "raw",
          heif_file.mode,
          heif_file.stride,
          )
      open_cv_image = np.array(image) 
      # Convert RGB to BGR 
      open_cv_image = open_cv_image[:, :, ::-1].copy() 
      cv2.imwrite(ORG_DIR + file_name.split(".")[0] + ".png", open_cv_image)
      base_names = [file_name.split(".")[0] + ".png"]
    else:
      img = cv2.imread(file_name)
      cv2.imwrite(ORG_DIR + file_name, img)
      base_names = [file_name]

  print("Running Engine...")
  run_engine(base_names)
  print("Finalizing images...")
  final_imgs(base_names)
  if is_video:
    print("Generating video...")
    create_vid(base_names, vidcap, save_name)
  else:
    img = cv2.imread(FINAL_DIR + base_names[0])
    cv2.imwrite(FINISHED_PRODUCTS + save_name + ".png", img)
  remove_temp()
  command = "rm " + file_name
  subprocess.call(command, shell=True)

Stylizing IMG_6296.mov
Extracting Frames...
Running Engine...


  0%|          | 0/238 [00:00<?, ?it/s]

Finalizing images...


  1%|          | 2/238 [00:00<00:15, 15.10it/s]

Generating video...


100%|██████████| 238/238 [00:16<00:00, 14.70it/s]
